# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [9]:
import numpy as np
import pandas as pd
import pymysql
from sqlalchemy import create_engine

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. Please, load the data using Ironhack's database (db: employees, table: employees_advanced).

In [68]:
driver = 'mysql+pymysql'
ip = '34.65.10.136'
username = 'data-students'
password = 'iR0nH@cK-D4T4B4S3'
db = 'employees'
connection_string  = f'{driver}://{username}:{password}@{ip}/{db}'
engine = create_engine(connection_string)

query = 'SELECT * from employees_advanced'
employees = pd.read_sql(query,engine)


Examine the `salaries` dataset using the `head` function below.

In [69]:
employees.head(10)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86
5,"ABBASI, CHRISTOPHER",STAFF ASST TO THE ALDERMAN,CITY COUNCIL,F,Salary,NaN,50436.0,NaN
6,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,F,Hourly,40.0,NaN,46.10
7,"ABBATE, JOSEPH L",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
8,"ABBATEMARCO, JAMES J",FIRE ENGINEER-EMT,FIRE,F,Salary,NaN,103350.0,NaN
9,"ABBATE, TERRY M",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [70]:
employees.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
Name                 33183 non-null object
Job Titles           33183 non-null object
Department           33183 non-null object
Full or Part-Time    33183 non-null object
Salary or Hourly     33183 non-null object
Typical Hours        8022 non-null float64
Annual Salary        25161 non-null float64
Hourly Rate          8022 non-null float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


#### There is 25161/33183 entry not nulls from 'Annual Salary', and 8022/33183 from 'Hourly Rate'

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [79]:
employees['Salary or Hourly'].value_counts()

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [91]:
dpmnt =  employees['Job Titles'].value_counts()
dpmnt.head()

POLICE OFFICER                            9520
FIREFIGHTER-EMT                           1473
SERGEANT                                  1202
POOL MOTOR TRUCK DRIVER                   1055
POLICE OFFICER (ASSIGNED AS DETECTIVE)     989
Name: Job Titles, dtype: int64

In [92]:
std_hours = employees['Hourly Rate'].std()

'''we use for challenge 2'''

12.11257268427681

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

-Confidence level = 95%

NULL HYPHOTESIS:$\;\;\;H_0 --> \mu_0 = 30 \$/h$

ALTERNATIVE HYPHOTESIS:$\;\;\;H_1 --> \mu_0 \ne 30 \$/h$  <-- *WE HAVE TO TEST IT!*

X = 32.7 $\$/h$ --> we take this value to recejt the null hypothesis

n = 8022 (people who work of hourly)

s= 12.1 --> std_hours

t(95\% for n>1000) = 1.96

CONFIDENCE INTERVAL

$[X-{t \cdot s \over n^(1/2)}],[X+{t \cdot s \over n^(1/2)}] = [32.7-0.265,\;\;32.7+0.265] = [32.535,\;\;32.965]$

**Use scipy. stats. ttest_1samp**

In [101]:
from scipy.stats import ttest_1samp

In [119]:
##remove the NaN vaues and reset index again.
hourly = employees['Hourly Rate'].dropna().reset_index(drop=True)

In [136]:
t, prob = ttest_1samp(hourly, 30)

In [137]:
print(f'''
T-stat: {t} 
Probability(p-Value):{prob}
''')


T-stat: 20.6198057854942 
Probability(p-Value):4.3230240486229894e-92



**Null hyphothesis is rejected because T-stat is too large to be accepted and probability to get this Hours rate payroll is minimum.**

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

-Confidence level = 95%

NULL HYPHOTESIS:$\;\;\;H_0 --> \mu_0 \leq 86000 \$$

ALTERNATIVE HYPHOTESIS:$\;\;\;H_1 --> \mu_0 > 86000 \$$  <-- *WE HAVE TO TEST IT!*

X = 32.7 $\$/h$ --> we take this value to recejt the null hypothesis

n = 8022 (people who work of hourly)

s= 12.1 --> std_hours

t(95\% for n>1000) = 1.96

CONFIDENCE INTERVAL

$[X-{t \cdot s \over n^(1/2)}],[X+{t \cdot s \over n^(1/2)}] = [32.7-0.265,\;\;32.7+0.265] = [32.535,\;\;32.965]$

**Use scipy. stats. ttest_1samp**

In [173]:
# Your code here:

salary = employees[employees.Department =='POLICE']['Annual Salary'].dropna().reset_index(drop=True)
salary.head()
t, prob = ttest_1samp(salary, 86000)
print(f'''
T-stat: {t} 
Probability(p-Value):{prob/2}
''')

print('if T-stat > 1.95 (z for 95% self confidence) we reject completely')


T-stat: 3.081997005712994 
Probability(p-Value):0.0010301701775482569

if T-stat > 1.95 (z for 95% self confidence) we reject completely


Using the `crosstab` function, find the department that has the most hourly workers. 

In [179]:
# Your code here:
pd.crosstab(employees['Department'], employees['Salary or Hourly']).sort_values('Hourly', ascending =  False).head()


Salary or Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336
WATER MGMNT,1513,366
OEMC,1273,829
AVIATION,1082,547
GENERAL SERVICES,765,215


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

-Confidence level = 95%

NULL HYPHOTESIS:$\;\;\;H_0 --> \mu_0 < 35 \$/hour$

ALTERNATIVE HYPHOTESIS:$\;\;\;H_1 --> \mu_0 > 35 \$/hour$  <-- *WE HAVE TO TEST IT!*

X = 32.7 $\$/h$ --> we take this value to recejt the null hypothesis


t(95\% for n>1000) = 1.96

CONFIDENCE INTERVAL

$[X-{t \cdot s \over n^(1/2)}],[X+{t \cdot s \over n^(1/2)}] $

**Use scipy. stats. ttest_1samp**

In [184]:
# Your code here:

hourly_streets = employees[employees.Department =='STREETS & SAN']['Hourly Rate'].dropna().reset_index(drop=True)
hourly_streets.head()
t, prob = ttest_1samp(hourly_streets, 35)
print(f'''
T-stat: {t} 
Probability(p-Value):{prob}
''')

print('if T-stat > 1.95 (z for 95% self confidence) we reject completely')


T-stat: -9.567447887848152 
Probability(p-Value):3.3378530564707717e-21

if T-stat > 1.95 (z for 95% self confidence) we reject completely


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [200]:
from scipy.stats import t

##interval(alpha, df, loc=0, scale=1)
            #self-confidence, number data, mean, std

mean = np.mean(hourly)
std = np.std(hourly)
df = hourly.count()-1
print(mean, std,df)

32.78855771628128 12.111817701093837 8021


In [195]:
t.interval(0.95, df,mean,std )


(9.046248988669632, 56.53086644389293)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [199]:
# Your code here:

from scipy.stats import t

##interval(alpha, df, loc=0, scale=1)
            #self-confidence, number data, mean, std

mean = np.mean(salary)
std = np.std(salary)
df = salary.count()
print(mean,std,df)
t.interval(0.95, df,mean,std )


86486.41450313339 18271.546685177447 13404


(50671.607030272746, 122301.22197599403)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [13]:
# Your code here:

